###############################################################################
# SRSWTITextAnalyzer
###############################################################################

In [ ]:
# ---------------------------  Imports -------------------------------
from srswti_axis import SRSWTITextAnalyzer
from utils import call_groq_llm, SAMPLE_DOCS
# import nltk
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('maxent_ne_chunker_tab')
# nltk.download('words')


/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 14:36:21.501176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741597581.521085  893328 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741597581.526598  893328 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 14:36:21.547328: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

# --------------------------- Basic  Demos -------------------------

In [2]:
# Initialize the SRSWTI text analyzer
core_analyzer = SRSWTITextAnalyzer()

INFO:SRSWTI:Initializing SRSWTI Text Analyzer
INFO:SRSWTI:SRSWTI Text Analyzer initialized with standard grammar patterns


In [6]:
# 1) Basic structure + sentiment
text_core_1 = """
Apple Inc. reported strong quarterly results. 
The company's innovation soared, but new regulatory hurdles might slow future growth.
"""
analysis_1 = core_analyzer.analyze_text(text_core_1)
print("== Core Analysis Example 1 ==")
print(analysis_1, "\n")


INFO:SRSWTI:Starting SRSWTI text analysis
INFO:SRSWTI:Completed analysis of 2 sentences


== Core Analysis Example 1 ==
{'summary': {'text_length': 134, 'sentence_count': 2, 'overall_sentiment': {'compound': 0.57375, 'pos': 0.37, 'neg': 0.0, 'neu': 0.63}, 'entity_count': 2, 'unique_entities': 2}, 'sentiment': {'overall': {'compound': 0.57375, 'pos': 0.37, 'neg': 0.0, 'neu': 0.63}, 'by_sentence': [{'text': '\nApple Inc. reported strong quarterly results.', 'scores': {'neg': 0.0, 'neu': 0.602, 'pos': 0.398, 'compound': 0.5106}}, {'text': "The company's innovation soared, but new regulatory hurdles might slow future growth.", 'scores': {'neg': 0.0, 'neu': 0.658, 'pos': 0.342, 'compound': 0.6369}}]}, 'entities': [{'text': 'Apple', 'label': 'PERSON', 'confidence': 0.85, 'position': {'start': 0, 'end': 0}}, {'text': 'Inc.', 'label': 'ORGANIZATION', 'confidence': 0.85, 'position': {'start': 0, 'end': 0}}], 'structure': {'phrases': {'noun_phrases': ['strong quarterly results', 'The company', 'innovation', 'new regulatory hurdles', 'future growth'], 'verb_phrases': ['reported strong

In [7]:
# 2) Named Entity Recognition & chunk-level sentiment
text_core_2 = """
QuantumCorp, a startup from Germany, secured $500 million in funding 
to develop a 5000-qubit quantum computer by 2026. 
"""
analysis_2 = core_analyzer.analyze_text(text_core_2)
print("== Core Analysis Example 2, Named Entities ==")
print(analysis_2.get("entities", []), "\n")

INFO:SRSWTI:Starting SRSWTI text analysis
INFO:SRSWTI:Completed analysis of 1 sentences


== Core Analysis Example 2, Named Entities ==
[{'text': 'QuantumCorp', 'label': 'GPE', 'confidence': 0.85, 'position': {'start': 0, 'end': 0}}, {'text': 'Germany', 'label': 'GPE', 'confidence': 0.85, 'position': {'start': 0, 'end': 0}}] 



In [8]:

# 2) Named Entity Recognition & chunk-level sentiment
text_core_2 = """
QuantumCorp, a startup from Germany, secured $500 million in funding 
to develop a 5000-qubit quantum computer by 2026. 
"""
analysis_2 = core_analyzer.analyze_text(text_core_2)
print("== Core Analysis Example 2, Named Entities ==")
print(analysis_2.get("entities", []), "\n")


INFO:SRSWTI:Starting SRSWTI text analysis
INFO:SRSWTI:Completed analysis of 1 sentences


== Core Analysis Example 2, Named Entities ==
[{'text': 'QuantumCorp', 'label': 'GPE', 'confidence': 0.85, 'position': {'start': 0, 'end': 0}}, {'text': 'Germany', 'label': 'GPE', 'confidence': 0.85, 'position': {'start': 0, 'end': 0}}] 



In [9]:
# 3) Using the structural breakdown to feed an LLM prompt
text_core_3 = """
In a surprising turn of events, the CFO resigned abruptly, 
citing personal reasons and health concerns.
"""
analysis_3 = core_analyzer.analyze_text(text_core_3)
print("== Core Analysis Example 3, structure for LLM prompt ==")
print(analysis_3.get("structure", {}), "\n")

INFO:SRSWTI:Starting SRSWTI text analysis
INFO:SRSWTI:Completed analysis of 1 sentences


== Core Analysis Example 3, structure for LLM prompt ==
{'phrases': {'noun_phrases': ['a surprising turn', 'events', 'personal reasons', 'health', 'concerns'], 'verb_phrases': ['citing personal reasons'], 'prep_phrases': ['In a surprising turn', 'of events']}, 'by_sentence': [{'text': '\nIn a surprising turn of events, the CFO resigned abruptly, \nciting personal reasons and health concerns.', 'structure': {'noun_phrases': ['a surprising turn', 'events', 'personal reasons', 'health', 'concerns'], 'verb_phrases': ['citing personal reasons'], 'prep_phrases': ['In a surprising turn', 'of events'], 'adj_phrases': []}}]} 



# -------------------- RAG Demos ----------------------

In [10]:
def rag_demo_a(doc):
    """
    Analyze a doc's structure & entities, feed to Groq LLM.
    """
    analysis = core_analyzer.analyze_text(doc)
    prompt = (
        f"Analyzed doc:\n{doc}\n\n"
        f"Entities: {analysis.get('entities', [])}\n"
        f"Structure: {analysis.get('structure', {})}\n"
        "Please provide a short summary or insight."
    )
    llm_response = call_groq_llm(prompt)
    print("[Theorem2 RAG Demo A] LLM Response:\n", llm_response, "\n")

def rag_demo_b(user_query, docs=SAMPLE_DOCS):
    """
    1) Analyze each doc
    2) 'Retrieve' docs that mention certain keywords
    3) Summarize with Groq
    """
    if not docs:
        print("[Theorem2 RAG Demo B] No docs.")
        return

    retrieved = []
    for d in docs:
        if "Neural" in d or "Reinforcement" in d:
            analysis = core_analyzer.analyze_text(d)
            retrieved.append((d, analysis.get("entities", None)))

    prompt = (
        f"User Query: {user_query}\n\n"
        f"Retrieved docs: {retrieved}\n"
        "Create a combined answer or summary."
    )
    llm_response = call_groq_llm(prompt)
    print("[Theorem2 RAG Demo B] LLM Response:\n", llm_response, "\n")



In [11]:
if SAMPLE_DOCS:
    doc_for_demo2 = SAMPLE_DOCS[0]
    rag_demo_a(doc_for_demo2)


INFO:SRSWTI:Starting SRSWTI text analysis
INFO:SRSWTI:Completed analysis of 10 sentences


[Theorem2 RAG Demo A] LLM Response:
 [Groq LLM] The analyzed document provides an overview of neural networks, a fundamental concept in artificial intelligence. Neural networks are computational models inspired by the human brain's architecture, consisting of interconnected nodes (neurons) organized in layers. They can learn and adapt through training processes, making them a crucial tool for various applications such as image and speech recognition, natural language processing, and medical diagnosis. Despite challenges like overfitting and interpretability, neural networks continue to drive innovations in numerous fields. The document highlights key concepts, types, and applications of neural networks, providing a comprehensive introduction to this complex topic. 



In [12]:
rag_demo_b("Tell me about neural networks or RL", SAMPLE_DOCS)


INFO:SRSWTI:Starting SRSWTI text analysis
INFO:SRSWTI:Completed analysis of 10 sentences


[Theorem2 RAG Demo B] LLM Response:
 [Groq LLM] Neural networks are computational models inspired by the human brain's architecture, consisting of interconnected nodes (neurons) organized in layers: input, hidden, and output layers. The key concepts of neural networks include their structure, where neurons receive inputs, apply weights, sum them, and pass through activation functions, and learning, where networks learn by adjusting weights through training processes like backpropagation. Activation functions, such as ReLU, sigmoid, or tanh, determine neuron output.

There are several types of neural networks, including feedforward neural networks, convolutional neural networks (CNNs), recurrent neural networks (RNNs), and transformers. Feedforward neural networks allow information to flow in one direction, while CNNs are specialized for image processing, RNNs handle sequential data with memory, and transformers are an advanced architecture for natural language processing.

Neural netwo